In [ ]:
# SOURCE
SOURCE = "https://www.tesladeaths.com"


In [ ]:
from requests_html import HTMLSession, Element

session = HTMLSession()
r: session = session.get(SOURCE)


In [ ]:
table: Element = r.html.find("#dttable", first=True)
table


In [ ]:
table_headers = table.find("th")
table_headers = [row.text for row in table_headers]
table_headers


In [ ]:
table_data = table.find("td")[:-350]
# * The last 350 td"s do not have useful data

for data in table_data:
    print(data.text)


In [ ]:
table_links = table.find("a")
urls = [link.attrs["href"] for link in table_links]
urls


In [ ]:
rows = []
for i in list(range(0, len(table_data), 23)):
    one_row = [data.text for data in table_data[0 + i : 23 + i]]
    rows.append(one_row)

for url in urls:
    for row in rows:
        row[17] = url

rows


In [ ]:
import pandas as pd

df = pd.DataFrame(rows, columns=table_headers)
df.to_csv("./data.csv")
df

# Was just trying out how Polars look like
# Polars store data in columnar format
# Which is why aggregation and stuff are lightning fast
# import polars as pl
# pl_rows = pl.DataFrame(rows)
# pl_rows


In [ ]:
# Converting necessary columns from str to int values
int_value_columns = df.columns[6:15]
df[int_value_columns] = (
    df[int_value_columns].apply(pd.to_numeric, errors="coerce").fillna(0).astype("int")
)
